### 1\. Что такое `psql`?

`psql` — это интерактивный терминал для PostgreSQL. Но называть его просто "терминалом" — значит преуменьшать его возможности. **`psql` — это швейцарский нож для администрирования, разработки, отладки и автоматизации задач** в PostgreSQL. Он позволяет не только выполнять SQL-запросы, но и использовать мета-команды, управлять выводом, писать скрипты и настраивать рабочее окружение под себя.

-----

### 2\. Основы: Подключение и Навигация

#### Подключение к базе данных

SQL Shell (psql) устонавливается вместе с СУБД. В моем сулчае вместе с PostgreSQL 17

* Существует проблема с обработкой кириллицы, что бы навсегда починить кодировку, посмотрите гайд  - (https://www.youtube.com/watch?v=nxGhGQFk34Y)
* Что бы `временно` (для текущей сессии) починить вывод рус. символов можно импользовать команду `plql \! chcp 1251` но после перезапуска терминала предётся заново вводить команду 

Самый распространенный способ запустить базу данных через `psql` — указать параметры подключения через флаги в командной строке:

  * `-h`: хост (адрес сервера)
  * `-U`: пользователь (user)
  * `-d`: имя базы данных (database)
  * `-p`: порт (по умолчанию 5432)
  * `-W`: принудительно запросить пароль

**Пример:** Подключимся к базе данных `northwind` на локальном сервере от имени пользователя `postgres`.

1) Через командную строку bash:
    ```bash
    psql -h localhost -U postgres -d northwind -W
    ```

2) Через SQL Shell (psql) - для windows: Нужно заполнить поля следующим образом:
    ```
    Server [localhost]:

    Database [postgres]: northwind

    Port [5432]:

    Username [postgres]:

    Пароль пользователя postgres:
     ```

После входа в `psql` вы можете переключаться между базами данных или переподключаться с другими учетными данными с помощью мета-команды `\c`.

```sql
-- Переключиться на базу данных 'template1' от имени пользователя 'admin'
\c template1 admin
```

Основные команды в `psql` (интерактивный терминал PostgreSQL) можно разделить на две категории: SQL-команды и мета-команды.

---

#### Навигация по базе данных

Вот базовые команды и мета-команды (`\` команды), которые нужно знать:

* **`CREATE DATABASE имя_базы;`**: Создает новую базу данных.
* **`DROP DATABASE имя_базы;`**: Удаляет существующую базу данных.

* **`\l`**: Выводит список всех баз данных на сервере.
* **`\c имя_базы`**: Подключается к указанной базе данных.
* **`\d`**: Показывает список всех таблиц, представлений и других объектов в текущей базе данных.
* **`\dt`**: Выводит список только таблиц.
* **`\d имя_таблицы`**: Показывает структуру (схему) указанной таблицы, включая столбцы, типы данных и индексы.
* **`\du`**: Выводит список всех пользователей (ролей) в системе.
* **`\q`**: Выходит из `psql`.
* **`\?`**: Показывает список всех мета-команд с кратким описанием.

**Пример использования с `northwind`:**
```sql
-- Посмотреть список таблиц
northwind=# \dt
             List of relations
 Schema |       Name        | Type  |  Owner
--------+-------------------+-------+----------
 public | categories        | table | postgres
 public | customer_customer | table | postgres
 public | customer_demograp | table | postgres
 public | customers         | table | postgres
 public | employees         | table | postgres
 ... (и так далее)

-- Описать таблицу 'orders'
northwind=# \d orders
                                        Table "public.orders"
     Column      |           Type           | Collation | Nullable |                  Default
-----------------+--------------------------+-----------+----------+-------------------------------------------
 order_id        | smallint                 |           | not null | nextval('orders_order_id_seq'::regclass)
 customer_id     | bpchar                   |           |          |
 employee_id     | smallint                 |           |          |
 order_date      | date                     |           |          |
...
Indexes:
    "pk_orders" PRIMARY KEY, btree (order_id)
Foreign-key constraints:
    "fk_orders_customers" FOREIGN KEY (customer_id) REFERENCES customers(customer_id)
    "fk_orders_employees" FOREIGN KEY (employee_id) REFERENCES employees(employee_id)
    "fk_orders_shippers" FOREIGN KEY (ship_via) REFERENCES shippers(shipper_id)
```

#### Схема `public`

По умолчанию, если вы не указываете схему, PostgreSQL ищет объекты (таблицы, функции и т.д.) в схеме `public`. Именно поэтому команда `\dt` показывает таблицы из этой схемы, и вы можете писать `SELECT * FROM orders;` вместо `SELECT * FROM public.orders;`.

-----

### 3\. Ключевые мета-команды (`\` команды)

Мета-команды — это "суперспособности" `psql`. Они начинаются с обратного слэша (`\`) и не являются SQL-командами.

#### 🕵️‍♂️ Информационные команды

  * `\dn`: Список **схем** (schemas).
  * `\dv`: Список **представлений** (views).
  * `\df`: Список **функций** (functions).
  * `\du`: Список **пользователей и ролей**.
  * `\d+ table_name`: **Расширенное описание** таблицы, включая комментарии к столбцам, статистику и метод хранения.

**Пример:**

```sql
northwind=# \d+ products
                                                         Table "public.products"
      Column       |           Type           | Collation | Nullable |                     Default                      | Storage  | Stats target | Description
-------------------+--------------------------+-----------+----------+--------------------------------------------------+----------+--------------+-------------
 product_id        | smallint                 |           | not null | nextval('products_product_id_seq'::regclass)     | plain    |              |
 product_name      | character varying(40)    |           | not null |                                                  | extended |              |
 supplier_id       | smallint                 |           |          |                                                  | plain    |              |
 ...
```

#### ⚙️ Команды выполнения

  * `\i filename.sql`: **Выполнить команды из файла**. Это незаменимо для применения миграций или больших скриптов.
  * `\o filename.txt`: **Перенаправить весь вывод** (включая результаты запросов) в файл. Чтобы остановить, введите `\o` без аргументов.

#### 🎨 Форматирование вывода

  * `\x`: **Включить/выключить расширенный режим отображения**. Идеально для таблиц с большим количеством столбцов или длинными значениями. Результат выводится в формате "ключ: значение".
  * `\pset option value`: Управление форматированием таблиц.
      * `\pset border 2`: Установить красивую рамку для таблиц.
      * `\pset format unaligned`: Убрать выравнивание (полезно для скриптов).
      * `\pset null '(NULL)'`: Отображать `NULL` значения как строку `(NULL)`.

#### 📜 История и редактирование

  * `\s`: Показать **историю выполненных команд**.
  * `\e`: Открыть **последнюю выполненную команду** во внешнем редакторе (задается переменной окружения `$EDITOR`, например, `vim` или `nano`). После сохранения и закрытия редактора команда немедленно выполнится. Очень удобно для многострочных запросов.

-----

### 4\. Прочие возможности

#### Использование переменных

Вы можете определять переменные внутри сессии `psql` и использовать их в запросах. Это крайне полезно для параметризации скриптов.

  * `\set my_var 'ALFKI'`: Установить переменную.
  * `\unset my_var`: Удалить переменную.
  * Использование в запросе: `:'my_var'` (обратите внимание на двоеточие и кавычки).

**Пример:**

```sql
northwind=# \set customer_id 'ALFKI'
northwind=# SELECT order_id, order_date FROM orders WHERE customer_id = :'customer_id';
 order_id | order_date
----------+------------
    10643 | 1997-08-25
    10692 | 1997-10-03
    10702 | 1997-10-13
...
```

Можно также передавать переменные из командной строки с помощью флага `-v`.

```bash
psql -d northwind -U postgres -v customer_id='ALFKI' -c "SELECT * FROM orders WHERE customer_id = :'customer_id';"
```

#### ⏱️ Измерение времени выполнения запросов (`\timing`)

Для анализа производительности всегда включайте тайминг.

```sql
northwind=# \timing
Timing is on.
northwind=# SELECT COUNT(*) FROM order_details;
  count
---------
 518640
(1 row)

Time: 104.134 ms
```

#### Управление транзакциями

По умолчанию `psql` работает в режиме **auto-commit**: каждый оператор выполняется в своей собственной транзакции и немедленно фиксируется. Для выполнения нескольких операций в одной транзакции используйте стандартные команды SQL:

  * `BEGIN;`: Начать транзакцию.
  * `COMMIT;`: Зафиксировать изменения.
  * `ROLLBACK;`: Откатить изменения.

💡 **Совет:** Для отладки или безопасных изменений можно установить `\set AUTOCOMMIT off`. В этом режиме `psql` не будет автоматически фиксировать транзакции. Вам придется явно вызывать `COMMIT` или `ROLLBACK`.

#### Динамическое выполнение с `\gexec` 🚀

Это одна из самых мощных, но редко используемых команд. `\gexec` берет результат запроса и **выполняет каждую строку этого результата как SQL-команду**.

**Сценарий:** Вам нужно выдать права на чтение всех таблиц в схеме `public` пользователю `reporting_user`.

```sql
-- 1. Сначала мы генерируем нужные нам команды GRANT
northwind=# SELECT format('GRANT SELECT ON TABLE %I TO reporting_user;', table_name)
northwind-# FROM information_schema.tables
northwind-# WHERE table_schema = 'public';
                       format
----------------------------------------------------
 GRANT SELECT ON TABLE categories TO reporting_user;
 GRANT SELECT ON TABLE products TO reporting_user;
 GRANT SELECT ON TABLE customers TO reporting_user;
 ...

-- 2. Теперь добавляем \gexec, чтобы выполнить этот результат
northwind=# SELECT format('GRANT SELECT ON TABLE %I TO reporting_user;', table_name)
northwind-# FROM information_schema.tables
northwind-# WHERE table_schema = 'public' \gexec
GRANT
GRANT
GRANT
...
```

`\gexec` сэкономил нам массу времени на ручном написании команд.

#### Работа с форматами вывода для скриптов

При автоматизации часто требуется вывод в машинно-читаемом формате:

  * `-A` или `--no-align`: Невыровненный вывод, идеально для парсинга.
  * `--csv`: Вывод в формате CSV.
  * `-H` или `--html`: Вывод в формате HTML-таблицы.

**Пример экспорта в CSV:**

```bash
psql -d northwind -U postgres --csv -c "SELECT product_id, product_name, unit_price FROM products WHERE unit_price > 50" > expensive_products.csv
```

-----

### 5\. Скрипты и автоматизация

`psql` отлично подходит для использования в shell-скриптах.

  * `psql -f script.sql`: Выполнить SQL-скрипт из файла.
  * `psql -c "SQL-command"`: Выполнить одну команду и выйти.

Для чистого вывода в скриптах используйте флаги:

  * `-q` (`--quiet`): Тихий режим, убирает приветственные сообщения.
  * `-t` (`--tuples-only`): Только данные, без заголовков столбцов и итоговых строк.

**Пример bash-скрипта для проверки количества заказов за сегодня:**

```bash
#!/bin/bash

DB_NAME="northwind"
DB_USER="postgres"

# Получаем количество сегодняшних заказов. -t убирает заголовки
ORDER_COUNT=$(psql -d "$DB_NAME" -U "$DB_USER" -t -c "SELECT COUNT(*) FROM orders WHERE order_date = CURRENT_DATE;")

# Убираем лишние пробелы
ORDER_COUNT=$(echo $ORDER_COUNT | xargs)
echo "Проверка заказов за $(date +%Y-%m-%d)..."
if [ "$ORDER_COUNT" -gt 0 ]; then
    echo "✅ Найдено $ORDER_COUNT новых заказов."
else
    echo "⚠️ Сегодня еще не было заказов."
fi
```
-----

### 6\. Конфигурация и кастомизация (`~/.psqlrc`)

Чтобы не вводить одни и те же команды в каждой сессии, создайте файл `.psqlrc` в вашем домашнем каталоге. Команды из этого файла выполняются при каждом запуске `psql`.

**Пример полезного `~/.psqlrc`:**

```sql
-- Всегда показывать время выполнения запросов
\timing on

-- Отображать NULL значения более явно
\pset null '¤'

-- Настроить красивый промпт: user@database (статус транзакции) #
-- %n - user, %/ - database
-- %R - статус транзакции: ^ для начала, ! для ошибки, * для транзакции
-- %# - '>' для обычного пользователя, '#' для суперпользователя
\set PROMPT1 '%n@%/%R%# '

-- Сохранять регистр при автодополнении по Tab
\set COMP_KEYWORD_CASE preserve-lower

-- Улучшить внешний вид таблиц
\pset border 2

-- Включить расширенный вывод для широких таблиц автоматически
\x auto

-- Приветственное сообщение
\echo 'Добро пожаловать в psql! Ваш .psqlrc успешно загружен.'
```

-----

### 7\. Советы

🛡️ **Безопасность паролей:**

  * **Никогда** не указывайте пароль в командной строке или в скриптах.
  * Используйте флаг `-W`, чтобы `psql` всегда запрашивал пароль интерактивно.
  * Избегайте переменной окружения `PGPASSWORD`, так как она может быть видна другим пользователям системы.
  * **Правильный путь:** Используйте файл `~/.pgpass`. Он позволяет хранить учетные данные для разных серверов в безопасном файле с ограниченными правами доступа (`chmod 600 ~/.pgpass`).

**Воспроизводимость:**

  * Для сложных операций всегда пишите SQL-код в `.sql` файле и выполняйте его через `\i`. Это делает ваши действия воспроизводимыми и легко отслеживаемыми в системах контроля версий.
  * Используйте связку `\o` и `\i` для генерации отчетов.

**Обучение:**

  * Используйте `\set ECHO_HIDDEN on`. Эта команда заставляет `psql` показывать SQL-запросы, которые он выполняет "под капотом" для реализации мета-команд (например, какой `SELECT` выполняется для `\dt`). Это отличный способ узнать больше о системном каталоге PostgreSQL.